In [84]:
import torch
from torch.utils.data import Dataset, DataLoader,TensorDataset

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.preprocessing import MinMaxScaler
import torch.nn as nn
from sklearn.model_selection import train_test_split
import category_encoders as ce

In [85]:
df = pd.read_excel("house_price_Test.xlsx")

In [86]:
value_counts = df['District'].value_counts()
district_counts = {}

for value, count in value_counts.items():
    district_counts[value] = count

In [107]:
filtered_df = df[df["District"] == "Çankaya"]



filtered_df = filtered_df.drop(columns=['Neighborhood',"District"], axis=1)


X = filtered_df.iloc[:, :-1].values
Y = filtered_df["Price"].values/100000000

In [108]:
X.shape

(5656, 18)

In [124]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [110]:
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
Y_train_scaled = scaler.fit_transform(y_train)
Y_test_scaled = scaler.transform(y_test)

In [125]:
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float()
X_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).float()

In [126]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)


train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [136]:
class AdaptiveTanh(torch.nn.Module):
    def __init__(self, init_alpha, init_beta):
        super(AdaptiveTanh, self).__init__()
        self.alpha = torch.nn.Parameter(torch.tensor(init_alpha))
        self.beta = torch.nn.Parameter(torch.tensor(init_beta))
    def forward(self, x):
        return torch.tanh(self.alpha*x + self.beta) 



class Net(nn.Module):
    def __init__(self,a = 0.5,b = 0.5):
        super(Net,self).__init__()
  
        self.layers = nn.Sequential(
      nn.Linear(18,18),
      nn.LeakyReLU(),
      nn.Linear(18,8),
      nn.ReLU(),
      nn.Linear(8,4),
      nn.ReLU(),
      nn.Linear(4,2),
      nn.ReLU(),
      nn.Linear(2,1),
    )
    def forward(self, x):
        return self.layers(x)

In [137]:
model = Net()

In [ ]:
criterion = nn.MSELoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=0.01,momentum=0.9)

num_epochs = 1000

for epoch in range(num_epochs):
    model.train()  
    epoch_loss = 0.0
    
    for batch_idx, (inputs, targets) in enumerate(train_dataloader):
       
        optimizer.zero_grad()

      
        outputs = model(inputs)

       
        loss = criterion(outputs.squeeze(), targets)

      
        loss.backward()

       
        optimizer.step()

        
        epoch_loss += loss.item()
    
  
    avg_loss = epoch_loss / len(train_dataloader)
    
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")